In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

import sys
import math
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary

import datetime
from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score

In [2]:
from ce_generator import CE5min

In [3]:
# Check dataset sanity

n_train_data = 40000
ce5 = CE5min(n_train_data, 'train')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset()
print(ce_data.shape, ce_labels.shape, ae_labels.shape)
print(ce_labels[np.random.randint(n_train_data)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [4]:
# Generate and save training data

# _, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset()
train_data = ae_labels
train_labels = ce_labels
# train_dataset = CEDataset(ae_labels, ce_labels)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)

In [9]:
idx=854
train_data[idx], train_labels[idx]

(array([7, 7, 7, 7, 7, 7, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 4, 4, 4, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [10]:
np.save('./CE_dataset/nar_5min_full_train_data_{}.npy'.format(n_train_data), train_data)
np.save('./CE_dataset/nar_5min_full_train_labels_{}.npy'.format(n_train_data), train_labels)

In [12]:
# Check dataset sanity

n_test_data = 1000
ce5 = CE5min(n_test_data, 'test')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset()
print(ce_data.shape, ce_labels.shape, ae_labels.shape)
print(ce_labels[np.random.randint(n_test_data)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [13]:
# Generate and save test data

_, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset()
test_data = ae_labels
test_labels = ce_labels

In [14]:
np.save('./CE_dataset/nar_5min_full_test_data.npy', test_data)
np.save('./CE_dataset/nar_5min_full_test_labels.npy', test_labels)

In [17]:
idx=830
test_data[idx], test_labels[idx]

(array([8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [18]:
actions,action_labels

(['walk',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'flush_toilet',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk'],
 [7,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  4,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7])

In [38]:
sensor_test_data = np.load('./CE_dataset/ce5min_test_data.npy')
sensor_test_labels = np.load('./CE_dataset/ce5min_test_labels.npy')

In [41]:
sensor_train_data =np.load('./CE_dataset/ce5min_train_data_2000.npy')

In [42]:
sensor_train_data

(2000, 60, 128)

In [39]:
idx=0
sensor_test_data[idx], sensor_test_data[idx]

(array([[0.02952732, 0.7663239 , 0.        , ..., 0.04166073, 0.07221995,
         0.15755884],
        [0.        , 0.7742814 , 0.        , ..., 0.12734969, 0.0987526 ,
         0.2630577 ],
        [0.        , 0.55041856, 0.00659789, ..., 0.21991117, 0.127577  ,
         0.4599582 ],
        ...,
        [0.        , 0.76084113, 0.        , ..., 0.11863101, 0.10553318,
         0.25677156],
        [0.        , 0.        , 0.05281916, ..., 0.02155555, 0.27263078,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.03247079, 0.23547615,
         0.10722303]], dtype=float32),
 array([[0.02952732, 0.7663239 , 0.        , ..., 0.04166073, 0.07221995,
         0.15755884],
        [0.        , 0.7742814 , 0.        , ..., 0.12734969, 0.0987526 ,
         0.2630577 ],
        [0.        , 0.55041856, 0.00659789, ..., 0.21991117, 0.127577  ,
         0.4599582 ],
        ...,
        [0.        , 0.76084113, 0.        , ..., 0.11863101, 0.10553318,
         0.25677156

In [22]:
# Check dataset sanity

n_train_data = 2000
ce5 = CE5min(n_train_data, 'train')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset()
print(ce_data.shape, ce_labels.shape, ae_labels.shape)
print(ce_labels[np.random.randint(n_train_data)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [23]:
# Generate and save training data

ce_data, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset()
train_data = ce_data
train_labels = ce_labels

# train_dataset = CEDataset(ae_labels, ce_labels)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)

In [24]:
np.save('./CE_dataset/ce5min_full_train_data_{}.npy'.format(n_train_data), train_data)
np.save('./CE_dataset/ce5min_full_train_labels_{}.npy'.format(n_train_data), train_labels)

In [25]:
# Check dataset sanity

n_test_data = 1000
ce5 = CE5min(n_test_data, 'test')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset()
print(ce_data.shape, ce_labels.shape, ae_labels.shape)
print(ce_labels[np.random.randint(n_test_data)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [26]:
# Generate and save test data
ce_data, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset()
test_data = ce_data
test_labels = ce_labels
test_ae_labels = ae_labels

In [27]:
np.save('./CE_dataset/ce5min_full_test_data.npy', test_data)
np.save('./CE_dataset/ce5min_full_test_labels.npy', test_labels)
np.save('./CE_dataset/ce5min_full_test_ae_labels.npy', test_ae_labels)

In [37]:
id = 0
test_ae_labels[id], test_labels[id]

(array([7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [ ]:
([7, 7, 7, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7])
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [30]:
# Check dataset sanity

n_val_data = 2000
ce5 = CE5min(n_val_data, 'test')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

# ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset(is_positive_sample=True)
# print(ce_data.shape, ce_labels.shape, ae_labels.shape)
# print(ce_labels[np.random.randint(n_val_data)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'

In [31]:
# Generate and save test data
ce_data, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset(is_positive_sample=False)

In [33]:
id = np.random.randint(n_val_data)
print(ce_data[id], ce_labels[id], ae_labels[id])
ce_data.shape

[[0.         2.4582243  0.         ... 0.96414465 0.63555396 1.0909039 ]
 [0.         0.04461792 0.         ... 0.07444356 0.28233784 0.09164882]
 [0.         0.         0.         ... 0.64825934 0.         0.27918133]
 ...
 [0.         0.17367312 0.         ... 0.09695865 0.18451798 0.3058947 ]
 [0.         0.7909916  0.         ... 0.27464926 0.10857708 0.378227  ]
 [0.         0.17902881 0.         ... 0.10151616 0.26368335 0.15831871]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [8 0 0 0 0 0 0 0 0 0 0 0 0 8 8 8 8 8 8 8 8 8 8 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]


(2000, 60, 128)

In [34]:
np.save('./CE_dataset/nar_5min_full_val_data.npy'.format(n_val_data), ae_labels)
np.save('./CE_dataset/nar_5min_full_val_labels.npy'.format(n_val_data), ce_labels)

In [35]:
n_val_data = 1000
ce5 = CE5min(n_val_data, 'test')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [36]:
# Generate and save test data
ce_data, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset(is_positive_sample=True)

In [38]:
id = np.random.randint(n_val_data)
print(ce_data[id], ce_labels[id], ae_labels[id])
ce_data.shape

[[0.         0.5359884  0.         ... 0.0745766  0.02610116 0.20467211]
 [0.         0.56582963 0.         ... 0.4599753  0.07268997 0.39582428]
 [0.         0.18522474 0.01741908 ... 0.20685308 0.25161624 0.31065935]
 ...
 [0.         0.         0.         ... 0.46242547 0.         0.34835932]
 [0.         0.9476439  0.         ... 0.5458216  0.40882888 0.44383428]
 [0.         0.90370625 0.         ... 0.29288423 0.39087576 0.0742722 ]] [0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [7 7 7 7 7 7 7 7 7 7 7 3 3 5 5 3 5 3 5 3 5 3 3 5 5 3 5 5 3 3 5 5 3 3 5 3 5
 3 5 5 3 5 5 3 3 5 3 5 5 3 3 5 3 3 5 5 3 3 5 5]


(1000, 60, 128)

In [39]:
np.save('./CE_dataset/ce5min_val_data.npy'.format(n_val_data), ce_data)
np.save('./CE_dataset/ce5min_val_labels.npy'.format(n_val_data), ce_labels)